In [1]:
%env SPARK_HOME=/usr/hdp/current/spark2-client
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print('Spark je pripraven ...')

env: SPARK_HOME=/usr/hdp/current/spark2-client
Spark je pripraven ...


In [ ]:
Nejprve zjistime zakladni informace o soboru kde jsou ulozena jednotliva hodnoceni. 

In [13]:
ratings = spark.read.format("csv")\
    .option("sep",";")\
    .option("header", "true")\
    .load("/data/books/BX-Book-Ratings.csv")

ratings.printSchema()

print("Pocet hodnoceni = %i" % ratings.count())

books = spark.read.format("csv")\
    .option("sep",";")\
    .option("header", "true")\
    .load("/data/books/BX-Books.csv")

books.printSchema()

print("Pocet knih = %i" % books.count())

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)

Pocet hodnoceni = 1149780
root
 |-- ISBN: string (nullable = true)
 |-- Book-Title: string (nullable = true)
 |-- Book-Author: string (nullable = true)
 |-- Year-Of-Publication: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Image-URL-S: string (nullable = true)
 |-- Image-URL-M: string (nullable = true)
 |-- Image-URL-L: string (nullable = true)

Pocet knih = 271379


In [14]:
from pyspark.sql.functions import col

ratings.groupBy("User-ID").count().orderBy(col("count").desc()).show(5)

+-------+-----+
|User-ID|count|
+-------+-----+
|  11676|13602|
| 198711| 7550|
| 153662| 6109|
|  98391| 5891|
|  35859| 5850|
+-------+-----+
only showing top 5 rows



In [26]:
ratings.groupBy("ISBN")\
    .count()\
    .join(books, ratings["ISBN"] == books["ISBN"], "left_outer")\
    .select("count", "Book-Title", "Book-Author", "Year-Of-Publication")\
    .orderBy(col("count").desc())\
    .show()

+-----+--------------------+---------------+-------------------+
|count|          Book-Title|    Book-Author|Year-Of-Publication|
+-----+--------------------+---------------+-------------------+
| 2502|         Wild Animus|   Rich Shapero|               2004|
| 1295|The Lovely Bones:...|   Alice Sebold|               2002|
|  883|   The Da Vinci Code|      Dan Brown|               2003|
|  732|Divine Secrets of...|  Rebecca Wells|               1997|
|  723|The Red Tent (Bes...|  Anita Diamant|               1998|
|  647|     A Painted House|   John Grisham|               2001|
|  639|                null|           null|               null|
|  615|The Secret Life o...|  Sue Monk Kidd|               2003|
|  614|Snow Falling on C...| David Guterson|               1995|
|  586| Angels &amp; Demons|      Dan Brown|               2001|
|  585|Where the Heart I...|   Billie Letts|               1998|
|  571|Harry Potter and ...|  J. K. Rowling|               1999|
|  568|The Pilot's Wife .